In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from snowflake.connector.pandas_tools import write_pandas, pd_writer

import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine


/home/sivachandan/anaconda3/envs/smoke/lib/python3.7/site-packages/snowflake/connector/options.py:99: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  _expected_pyarrow_version,


### Loading the Data

In [12]:
df_raw = pd.read_csv("./data/train_dataset.csv")
df_raw.size

896632

### Splitting the Data

In [3]:
train_data,test_data = train_test_split(df_raw,test_size=0.2,train_size=0.8)

### Training the model

In [4]:
model = RandomForestClassifier(n_estimators=100)
model.fit(train_data.iloc[:,:-1],train_data['smoking'])

RandomForestClassifier()

In [5]:
model.score(test_data.iloc[:,:-1],test_data['smoking'])

0.7987687572143132

In [6]:
print(pd.io.sql.get_schema(df_raw.reset_index(),'smoking_data'))

CREATE TABLE "smoking_data" (
"index" INTEGER,
  "age" INTEGER,
  "height(cm)" INTEGER,
  "weight(kg)" INTEGER,
  "waist(cm)" REAL,
  "eyesight(left)" REAL,
  "eyesight(right)" REAL,
  "hearing(left)" INTEGER,
  "hearing(right)" INTEGER,
  "systolic" INTEGER,
  "relaxation" INTEGER,
  "fasting blood sugar" INTEGER,
  "Cholesterol" INTEGER,
  "triglyceride" INTEGER,
  "HDL" INTEGER,
  "LDL" INTEGER,
  "hemoglobin" REAL,
  "Urine protein" INTEGER,
  "serum creatinine" REAL,
  "AST" INTEGER,
  "ALT" INTEGER,
  "Gtp" INTEGER,
  "dental caries" INTEGER,
  "smoking" INTEGER
)


/home/sivachandan/anaconda3/envs/smoke/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  """Entry point for launching an IPython kernel.


In [25]:
engine = create_engine(URL(user = os.getenv('SNOWSQL_USER'),\
    password = os.getenv('SNOWSQL_PASSWORD'),\
        account = os.getenv('SNOWSQL_ACCOUNT'),\
            database = 'smoking',schema = 'public',role = 'ACCOUNTADMIN',warehouse = 'compute_wh'))
conn = engine.connect()
# conn.execute('truncate table smoking')
results = conn.execute("SELECT current_version()").fetchone()
results[0]
conn.close()
engine.dispose()

In [19]:
df_chunks = pd.read_csv('./data/train_dataset.csv',chunksize=1000)
for i,df_chunk in enumerate(df_chunks):
    try:
        df_chunk.to_sql('smoking',con=conn,index=False,if_exists='append')
    except:
        df_chunk.to_sql('smoking',con=conn,index=False,if_exists='replace')

In [13]:

df_raw.to_sql('smoking',con=conn,index=False,if_exists='replace')

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 000606 (57P03): No active warehouse selected in the current session.  Select an active warehouse with the 'use warehouse' command.

[SQL: 
            SELECT /* sqlalchemy:_get_schema_columns */
                   ic.table_name,
                   ic.column_name,
                   ic.data_type,
                   ic.character_maximum_length,
                   ic.numeric_precision,
                   ic.numeric_scale,
                   ic.is_nullable,
                   ic.column_default,
                   ic.is_identity,
                   ic.comment,
                   ic.identity_start,
                   ic.identity_increment
              FROM information_schema.columns ic
             WHERE ic.table_schema=%(table_schema)s
             ORDER BY ic.ordinal_position]
[parameters: {'table_schema': 'PUBLIC'}]
(Background on this error at: https://sqlalche.me/e/14/f405)